# Deep dive into Tensorflow example “Partial Differential Equations”

## Problem statement

Tensorflow documentation provides very [nice tutorial examples](https://www.tensorflow.org/tutorials/). One of the non machine learning examples called ["Partial Differential Equations"](https://www.tensorflow.org/tutorials/non-ml/pdes), which models a surface of a pond as few raindrops land on it.

Tensorflow code of this example is straightforward, but mathematics and physics behind is not. I can only imagine present physicist students, who can say from top of their head on what is going on in this example. So far i'm pretty sure they still would spend some time refreshing some necessary knowledge.

I was a physicist student some time ago, so let me try to shed some light. As a discamiler i want to say that in some statements i potentially can be wrong, so far in general i have a sense of overall picture now (after 3 nights :) ).

![](https://github.com/r7vme/tensorflow-example-pdes/blob/master/example.jpeg?raw=true)

## Quick source code explanation

Original source code: https://www.tensorflow.org/tutorials/non-ml/pdes

Extra changes:
- added speed of the waves $c$
- Laplacian kernel divided by 2 to be aligned with kernel from Wikipedia and other papers

If you already know the code, then feel free to skip to [Mathematics](#Mathematics) section.

In this example author tries to model a physical process that happens when rain drop hit a water surface. Tensorflow framework mostly used here to do a parallel linear algebra computations. As you know Tensorflow has two things in it's core: tensor and operation. Tensor is usually some vector (matrix) or scalar. Operation can be some linear algebra operation (e.g. matrix multiplication). Both of them are part (nodes) of internal TensorFlow graph. This graph can be executed within a TensorFlow session.

So let's see what tensors and operations we need to model our physical process in TensorFlow.

First import necessary libraries and define function that will draw a pond.

In [ ]:
#Import libraries for simulation
import tensorflow as tf
import numpy as np

#Imports for visualization
import PIL.Image
from io import BytesIO
from IPython.display import clear_output, Image, display


def DisplayArray(a, fmt='jpeg', rng=[0,1]):
  """Display an array as a picture."""
  a = (a - rng[0])/float(rng[1] - rng[0])*255
  a = np.uint8(np.clip(a, 0, 255))
  f = BytesIO()
  PIL.Image.fromarray(a).save(f, fmt)
  clear_output(wait = True)
  display(Image(data=f.getvalue()))

Then initialize Tensorflow session. 

In [ ]:
sess = tf.InteractiveSession()

Following two functions are necessary to perform operation called convolution. Convolutional kernel `k` slides the image `x` and produces new matrix `y`. Every `y` value is a sum of multiplication of area (covered by `k`) on `x` and kernel `k`.

![](https://cdn-images-1.medium.com/max/1040/1*ZCjPUFrB6eHPRi4eyP6aaA.gif)
[image source](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1)

NOTE: In our case `y` will be the same shape as `x`, because padding set to `SAME`.

In [ ]:
def make_kernel(a):
  """Transform a 2D array into a convolution kernel"""
  a = np.asarray(a)
  a = a.reshape(list(a.shape) + [1,1])
  return tf.constant(a, dtype=1)

def simple_conv(x, k):
  """A simplified 2D convolution operation"""
  x = tf.expand_dims(tf.expand_dims(x, 0), -1)
  y = tf.nn.depthwise_conv2d(x, k, [1, 1, 1, 1], padding='SAME')
  return y[0, :, :, 0]

Discrete Laplace operator described below.

In [ ]:
def laplace(x):
  """Compute the 2D laplacian of an array"""
  laplace_k = make_kernel([[0.25, 0.5, 0.25],
                           [0.5, -3., 0.5],
                           [0.25, 0.5, 0.25]])
  return simple_conv(x, laplace_k)

Initialize pond `u_init` (500x500x1) with zeros. Also initialize matrix that will store first derivative `ut_unit`.

In [ ]:
N = 500

# Initial Conditions -- some rain drops hit a pond

# Set everything to zero
u_init = np.zeros([N, N], dtype=np.float32)
ut_init = np.zeros([N, N], dtype=np.float32)

# Some rain drops hit a pond at random points
for n in range(40):
  a,b = np.random.randint(0, N, 2)
  u_init[a,b] = np.random.uniform()

DisplayArray(u_init, rng=[-0.1, 0.1])

Define TensorFlow input, variables and finally two operations. Then group them and run session. All details below.

In [ ]:
# Parameters:
# eps -- time resolution
# damping -- wave damping
# c -- wave speed # Not a part of original code version
eps = tf.placeholder(tf.float32, shape=())
damping = tf.placeholder(tf.float32, shape=())
c = tf.placeholder(tf.float32, shape=())

# Create variables for simulation state
U  = tf.Variable(u_init)
Ut = tf.Variable(ut_init)

# Discretized PDE update rules
U_ = U + eps * Ut
Ut_ = Ut + eps * ((c ** 2) * laplace(U) - damping * Ut)

# Operation to update the state
step = tf.group(
  U.assign(U_),
  Ut.assign(Ut_))

# Initialize state to initial conditions
tf.global_variables_initializer().run()

Finally run 1000 steps and display pond after every step.

In [ ]:
# Run 1000 steps of PDE
for i in range(1000):
  # Step simulation
  step.run({eps: 0.03, damping: 0.04, c: 3.0})
  DisplayArray(U.eval(), rng=[-0.1, 0.1])

sess.close()

### Now feel free to run the code. Mathematical explanation is coming.

## Mathematics

Let's try formulate this code from mathematical point of view.

This example is a numerical solution ([Euler's method](https://en.wikipedia.org/wiki/Numerical_methods_for_ordinary_differential_equations#Euler_method)) of [wave equation](https://en.wikipedia.org/wiki/Wave_equation) (second order Partial Differencian Equation - PDE). It's quite hard to model PDEs (and computationary expensive?), that's why PDE was transformed into ODE (oridinary differential equation), which only has one differentiable variable - time. And can be easily computed with Euler's method.

Transformation of PDE to ODE is done with Laplacian transformation. In particular, [discrete Laplacian operator](https://en.wikipedia.org/wiki/Discrete_Laplace_operator) (very rough approximation of second derivative) was used. More precisely isotropic discrete Laplacian operator. Isotropic discrete Laplacian operator is just a 3x3 matrix (convolutional kernel) in our case.

### Wave equation with Euler's method

In general, this problem has analytical solutions, e.g. if we restrict our waves to be monochomatic (single frequency), cylindrical waves in cylindrical coordinate system ([example](http://farside.ph.utexas.edu/teaching/315/Waves/node47.html)). So instead of brute force approximation of second derivatives, we can compute bunch of sin/cos functions and then project cylindrical coordinates to euclidean. But this at least requires careful analytical equations preparation, which can take a time. So let's look on numerical solution instead.

We want to modify a wave function U(x,y), so it's values can be computed using Euler's method. For Euler we need first order differential equations, but wave function is a second order. This means we need to apply Euler's method twice.

Initial [wave equation](https://en.wikipedia.org/wiki/Wave_equation#Introduction) looks like

\begin{equation} 
 \frac{{\partial ^2 u}}{{\partial t^2 }} = {{c^2 \Delta u}}
\end{equation}

Which is a second direvative $u''$. Let's write discrete equations for $u$, where $\Delta t$ is a time difference

\begin{equation} 
 u_0(x,y) = C\\
 u_{n+1}(x,y) = u_n(x,y) + u'_n(x,y) \Delta t\\
 u'_{n+1}(x,y) = u'_n(x,y) + u''_n(x,y) \Delta t\\
\end{equation}

We know initial conditions $C$ and we also can replace $u''$ with right part of wave equation.

\begin{equation} 
 u'_{n+1}(x,y) = u'_n(x,y) + c^2 \Delta u \Delta t\\
\end{equation}

Let's also add reasoning behind wave dampling. Assuming this is a friction force, which for small speeds proportional to speed.

\begin{equation} 
 u'_{n+1}(x,y) = u'_n(x,y) + (c^2 \Delta u - k_{dampling} u'_n(x,y)) \Delta t\\
\end{equation}

Final form that is used in code is

\begin{equation}
 u_0(x,y) = C\\
 u_{n+1}(x,y) = u_n(x,y) + u'_n(x,y) \Delta t\\
 u'_{n+1}(x,y) = u'_n(x,y) + (c^2 \Delta u - k_{dampling} u'_n(x,y)) \Delta t\\
\end{equation}

### Discrete Laplacian operator

Laplacian can be a tough thing to understand actually. But let me try to explain in simple words. For detailed explanation look [here](https://ocw.mit.edu/courses/mathematics/18-03-differential-equations-spring-2010/video-lectures/lecture-19-introduction-to-the-laplace-transform/) (MIT lecture), for intuitions look [here](https://dsp.stackexchange.com/questions/11008/intuitive-interpretation-of-laplace-transform).

Small context about [Fourier transform](https://en.wikipedia.org/wiki/Fourier_transform) (easier intuition).

In our case we have some signal (wave), it was proven that **every signal can be broken into parts**. In case, of Fourier transform signal can be described as a **sum of harmonics (sinusoids)**. In other words, Fourier transform breaks **ANY** complex signal into bunch of simple sinusoids. Fourier transform is a special case of Laplace transform.

In case of Laplace transform signal can be described by it’s "moments" (i.e. mean and variance, first and second derivative respectively). Put simpler (for our case), we can extract enough information from neighbours spatial points to estimate our next value (i.e. move from spatial x,y variables to time variable only). Just to remember, Laplacian transformation allows partial differential equation (PDE) to become ordinary differential equation (ODE), which is much simpler to solve.

For numerical solutions discrete Laplace operator usually used with convolution. Most frequent use case is image edge detection. Basically we slide the image with special filter (kernel). Kernel is just a 3x3 matrix that is an rough approximation of analytical Laplace transform. Multiplying neighbour pixels values with kernel and then taking a sum of all values, gives us second derivative (i.e. speed of intensity/color change) that is then used in wave equation.

Plots of non-isotropic (left) and isotropic Laplacian kernels.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ee/Laplacetf.png/600px-Laplacetf.png)
[source wikipedia](https://de.wikipedia.org/wiki/Laplace-Filter)

Small note, author of Tensorflow example used kernel matrix that for some reason was multiplied by 2 (may be i'm missing smth?). According to [wikipedia](https://en.wikipedia.org/wiki/Discrete_Laplace_operator#Implementation_via_operator_discretization), isotropic Laplace convolution kernel is


\begin{bmatrix}
    0.25 & 0.5 & 0.25 \\
    0.5 & -3 & 0.5 \\
    0.25 & 0.5 & 0.25
\end{bmatrix}

One more time, **Laplacian kernel can be used to get rough approximation of the second partial derivative** of the function.

## Physical parameters of waves

Just a few notes about physical parameters of the waves (wave length, frequency). Because of numerical approximation limiations it seems impossible to get a single harmonic wave.

For example, single harmonic cylindrical wave should have constant frequency (constant distance between wave crests), but as experiments show distance is changing. I assume this is because that initial wave contains multiple harmonics, so eventually they are splitting up.

## Conclusion

In this blog post, i've described results of my small research regarding math background of Tensorflow example [Partial Differential Equations](https://www.tensorflow.org/tutorials/non-ml/pdes). As a result, i've added speed parameter and corrected Laplace kernel for isotropic case.